In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import csv
from collections import Counter 
from collections import *
import math
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [113]:
!pip3 install jupyternotify

  Created wheel for jupyternotify: filename=jupyternotify-0.1.15-py3-none-any.whl size=8720 sha256=b9819095f41b45c3f0292a91c36f6e86fa2949838a456999a520d6d5b7be9d44
  Stored in directory: /Users/compmusic/Library/Caches/pip/wheels/2d/ff/e5/bced088d6443a70f72ee86f9e46f11056c981100c2d3cfadfb
Successfully built jupyternotify


# Import Data from Google Sheets}

In [2]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [3]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# Reference: https://developers.google.com/sheets/api/quickstart/python
def read_google_sheets(SPREADSHEET_ID, RANGE_NAME, HEADER_RANGE):
    creds = None
    # autogenerated
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                range=RANGE_NAME).execute()
    
    header = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                               range=HEADER_RANGE).execute()
    
    header_values = header.get('values', [])
    values = result.get('values', [])
    
    return values, header_values

In [4]:
SPREADSHEET_ID = '1AjynK9mMQTw58B_B8b_ZIip3fyUm-aoV7Pp21HziBb0'
RANGE_NAME = 'canto_codings!A2:AT'
HEADER_RANGE = 'canto_codings!A1:AT1'

data, header = read_google_sheets(SPREADSHEET_ID, RANGE_NAME, HEADER_RANGE)

df = pd.DataFrame(data, columns = header[0])

In [5]:
df.head()

,canto_coding_id,Culture,C_cid,cv_1,cv_2,cv_3,cv_4,cv_5,cv_6,cv_7,...,cv_34,cv_35,cv_36,cv_37,orv_1,orv_2,ensemble_value_id,ensemble_value_label,instrument_value_id,instrument_value_label
0,1,South Jakarta,17557,4,512,64,16,2,2,9216,...,16,16,128,16,1,6,1,All singers play instruments,6,5 kinds of instruments
1,2,Balinese,11507,4,2,2,16,2,2,2,...,2,2,128,16,1,0,null,,null,None
2,3,Balinese,11507,512,8,64,128,16,128,8192,...,16,16,2,128,7,3,1,All singers play instruments,3,2 kinds of instruments
3,4,Balinese,11507,4096,4096,4096,128,128,8192,8192,...,16,2,1024,16,7,3,1,All singers play instruments,1,No instruments
4,5,Sama,62459,4,256,4,16,2,2,16,...,16,16,128,16,1,1,null,,1,No instruments


In [98]:
columns = ['index','soc_id', 'canto_coding_id', 'var_id', 'code']
long_form = pd.DataFrame(columns = columns)

In [116]:
df_1 = df[:1000]
df_2 = df[1000:2000]
df_3 = df[2000:3000]
df_4 = df[3000:4000]
df_5 = df[4000:]

In [105]:
def convert_matrix(input_matrix):
    index = 0
    output_matrix = pd.DataFrame(columns = columns)
    for i, row in input_matrix.iterrows():
        canto_coding_id = row['canto_coding_id']
        soc_id = row['C_cid']
        for j in range(37):
            var_id = j+1
            code = row['cv_'+str(var_id)]
            index +=1
            new_row = pd.DataFrame([[int(index),soc_id, int(canto_coding_id), int(var_id), int(code)]], columns=columns)
            output_matrix = output_matrix.append(new_row)
    return output_matrix

In [106]:
long_form = long_form.append(convert_matrix(df_1))

In [108]:
long_form = long_form.append(convert_matrix(df_2))

In [110]:
long_form = long_form.append(convert_matrix(df_3))

In [115]:
long_form = long_form.append(convert_matrix(df_4))

In [117]:
long_form = long_form.append(convert_matrix(df_5))

In [121]:
long_form = long_form.drop(['index'], axis=1)

In [122]:
long_form.to_csv('long_form.csv')